In [1]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np
from threading import Thread

def create_image(root, filename, x=0, y=0, width=None, height=None, bg=None):
    """
    Создает и размещает изображение в окне Tkinter
    
    Параметры:
        root: окно Tkinter (Tk() или Frame)
        filename: имя файла изображения (например 'tank.png')
        x, y: координаты размещения (по умолчанию 0, 0)
        width: желаемая ширина (None - оригинальный размер)
        height: желаемая высота (None - оригинальный размер)
        bg: цвет фона (None - прозрачный)
    
    Возвращает:
        Label: созданный виджет с изображением
    """
    # Загружаем изображение через Pillow
    pil_image = Image.open(filename)
    
    # Изменяем размер если нужно
    if width or height:
        original_width, original_height = pil_image.size
        
        # Если указана только одна размерность, сохраняем пропорции
        if not width:
            ratio = height / original_height
            width = int(original_width * ratio)
        elif not height:
            ratio = width / original_width
            height = int(original_height * ratio)
            
        pil_image = pil_image.resize((width, height), Image.LANCZOS)
    
    # Конвертируем в формат Tkinter
    tk_image = ImageTk.PhotoImage(pil_image)
    
    # Создаем Label с изображением
    label = Label(root, image=tk_image, bg=bg)
    label.image = tk_image  # Сохраняем ссылку на изображение!
    label.place(x=x, y=y)
    
    return label

from tkinter import font as tkfont  # Для работы с шрифтами

def create_text_label(root, text, x=0, y=0, 
                     font_size=12, font_family='Arial', font_weight='normal',
                     bg='white', fg='black', 
                     width=None, height=None,
                     anchor='nw'):
    """
    Создает текстовую метку с заданными параметрами
    
    Параметры:
        root: окно Tkinter (Tk или Frame)
        text: отображаемый текст
        x, y: координаты размещения (по умолчанию 0, 0)
        font_size: размер шрифта (по умолчанию 12)
        font_family: семейство шрифтов (по умолчанию Arial)
        font_weight: насыщенность (normal/bold)
        bg: цвет фона (по умолчанию white)
        fg: цвет текста (по умолчанию black)
        width: ширина метки (None - автоматически)
        height: высота метки (None - автоматически)
        anchor: выравнивание текста (nw, n, ne, w, center, e, sw, s, se)
    
    Возвращает:
        Label: созданный текстовый виджет
    """
    # Создаем объект шрифта
    custom_font = tkfont.Font(
        family=font_family,
        size=font_size,
        weight=font_weight
    )
    
    # Создаем метку
    label = Label(
        root,
        text=text,
        font=custom_font,
        bg=bg,
        fg=fg,
        width=width,
        height=height,
        anchor=anchor
    )
    
    # Размещаем метку
    label.place(x=x, y=y)
    
    return label

class CameraWidget:
    def __init__(self, parent, x=0, y=0):
        self.parent = parent
        self.camera_active = False
        self.video_capture = None
        
        # Создаем фрейм для камеры
        self.camera_frame = Frame(parent, bg='green')
        self.camera_frame.place(x=x, y=y)
        
        # Метка для вывода изображения
        self.camera_label = Label(self.camera_frame, bg='black')
        self.camera_label.pack()
        
        # Кнопка управления камерой
        self.btn_camera = Button(self.camera_frame, 
                               text='Включить камеру', 
                               command=self.toggle_camera,
                               font=('Arial', 10),
                               bg='blue',
                               fg='white')
        self.btn_camera.pack(pady=5)
    
    def toggle_camera(self):
        """Включение/выключение камеры"""
        if not self.camera_active:
            self.start_camera()
        else:
            self.stop_camera()
    
    def start_camera(self):
        """Запуск камеры"""
        self.camera_active = True
        self.btn_camera.config(text='Выключить камеру', bg='red')
        self.video_capture = cv2.VideoCapture(0)
        Thread(target=self.update_frame, daemon=True).start()
    
    def stop_camera(self):
        """Остановка камеры"""
        self.camera_active = False
        self.btn_camera.config(text='Включить камеру', bg='blue')
        if self.video_capture:
            self.video_capture.release()
        self.camera_label.config(image='')
    
    def update_frame(self):
        """Обновление кадров"""
        while self.camera_active:
            ret, frame = self.video_capture.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (180, 140))
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                
                self.camera_label.imgtk = imgtk
                self.camera_label.config(image=imgtk)
                
                self.parent.update()
                cv2.waitKey(30)
            else:
                break

class Signals:
    def __init__(self, root):
        self.root = root
        self.current_dron = None  # Текущее изображение дрона
        self.dron_img = self._load_image()  # Загружаем изображение один раз

    def _load_image(self):
        """Загружает и подготавливает изображение дрона"""
        try:
            img = Image.open('dron_atak.png')
            img = img.resize((70, 70), Image.LANCZOS)  # Фиксированный размер
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Ошибка загрузки изображения дрона: {e}")
            return None

    def dron_direction(self, direction):
        """Отображает атаку дрона с указанного направления"""
        # Удаляем предыдущее изображение
        if self.current_dron:
            self.current_dron.destroy()

        # Позиции для разных направлений
        positions = {
            'Front': {'x': 240, 'y': 160},
            'Right': {'x': 350, 'y': 320},
            'Left': {'x': 120, 'y': 320},
            'Back': {'x': 250, 'y': 500},
            'Up': {'x': 200, 'y': 100}
        }

        # Создаем новое изображение дрона
        if direction in positions and self.dron_img:
            pos = positions[direction]
            self.current_dron = Label(self.root, image=self.dron_img, bg='red')
            self.current_dron.place(x=pos['x'], y=pos['y'])

    def danger(self, many):
        """
        Отображает предупреждение об обнаруженных FPV-дронах
        
        Параметры:
            many: количество обнаруженных дронов (int)
        """
        # Удаляем предыдущие элементы, если они есть
        if hasattr(self, 'danger_label'):
            self.danger_label.destroy()
        if hasattr(self, 'fpv_label'):
            self.fpv_label.destroy()
        if hasattr(self, 'status_label'):
            self.status_label.destroy()
        
        # Создаем элементы предупреждения
        try:
            self.danger_label = create_image(self.root, 'dangeros.png', x=380, y=10, width=150, bg='snow')
            #self.fpv_label = create_image(self.root, 'fpv.jpeg', x=500, y=50, width=100)

            if many == 1:
                Text=f'Обнаружен {many} FPV-дрон!'
            elif 2<= many <=4:
                Text=f'Обнаружено {many} FPV-дрона!'
            else:
                Text=f'Обнаружено {many} FPV-дронов!'
            
            self.status_label = create_text_label(
                self.root,
                text=Text,
                x=325, y=100,
                font_size=14,
                bg='red',
                fg='white',
                font_weight='bold'
            )
        except Exception as e:
            print(f"Ошибка при создании элементов опасности: {e}")    
    

# Основное окно
root = Tk()
root.geometry('600x600')
root.resizable(False, False)
root.title('Система Щит с камерой')
root.config(bg='green')
root.iconbitmap('icon.ico')

# Виджет камеры
camera = CameraWidget(root, x=0, y=0)


# Изображение танка с размещением через place
tank = create_image(root, 'tank.png', x=150, y=150, width=300, bg='green')


signals = Signals(root)
signals.dron_direction('Front')
signals.danger(5)




root.mainloop()

KeyboardInterrupt: 

In [4]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np
from threading import Thread
from ultralytics import YOLO
import time
from tkinter import font as tkfont
import os

def create_image(root, filename, x=0, y=0, width=None, height=None, bg=None):
    """
    Создает и размещает изображение в окне Tkinter
    
    Параметры:
        root: окно Tkinter (Tk() или Frame)
        filename: имя файла изображения (например 'tank.png')
        x, y: координаты размещения (по умолчанию 0, 0)
        width: желаемая ширина (None - оригинальный размер)
        height: желаемая высота (None - оригинальный размер)
        bg: цвет фона (None - прозрачный)
    
    Возвращает:
        Label: созданный виджет с изображением
    """
    # Загружаем изображение через Pillow
    pil_image = Image.open(filename)
    
    # Изменяем размер если нужно
    if width or height:
        original_width, original_height = pil_image.size
        
        # Если указана только одна размерность, сохраняем пропорции
        if not width:
            ratio = height / original_height
            width = int(original_width * ratio)
        elif not height:
            ratio = width / original_width
            height = int(original_height * ratio)
            
        pil_image = pil_image.resize((width, height), Image.LANCZOS)
    
    # Конвертируем в формат Tkinter
    tk_image = ImageTk.PhotoImage(pil_image)
    
    # Создаем Label с изображением
    label = Label(root, image=tk_image, bg=bg)
    label.image = tk_image  # Сохраняем ссылку на изображение!
    label.place(x=x, y=y)
    
    return label

from tkinter import font as tkfont  # Для работы с шрифтами

def create_text_label(root, text, x=0, y=0, 
                     font_size=12, font_family='Arial', font_weight='normal',
                     bg='white', fg='black', 
                     width=None, height=None,
                     anchor='nw'):
    """
    Создает текстовую метку с заданными параметрами
    
    Параметры:
        root: окно Tkinter (Tk или Frame)
        text: отображаемый текст
        x, y: координаты размещения (по умолчанию 0, 0)
        font_size: размер шрифта (по умолчанию 12)
        font_family: семейство шрифтов (по умолчанию Arial)
        font_weight: насыщенность (normal/bold)
        bg: цвет фона (по умолчанию white)
        fg: цвет текста (по умолчанию black)
        width: ширина метки (None - автоматически)
        height: высота метки (None - автоматически)
        anchor: выравнивание текста (nw, n, ne, w, center, e, sw, s, se)
    
    Возвращает:
        Label: созданный текстовый виджет
    """
    # Создаем объект шрифта
    custom_font = tkfont.Font(
        family=font_family,
        size=font_size,
        weight=font_weight
    )
    
    # Создаем метку
    label = Label(
        root,
        text=text,
        font=custom_font,
        bg=bg,
        fg=fg,
        width=width,
        height=height,
        anchor=anchor
    )
    
    # Размещаем метку
    label.place(x=x, y=y)
    
    return label


# Класс для работы с сигналами
class Signals:
    def __init__(self, root):
        self.root = root
        self.current_dron = None
        self.dron_img = self._load_image()

    def _load_image(self):
        """Загружает изображение дрона"""
        try:
            img = Image.open('dron_atak.png')
            img = img.resize((70, 70), Image.LANCZOS)
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Ошибка загрузки изображения дрона: {e}")
            return None

    def dron_direction(self, direction):
        """Отображает атаку дрона с указанного направления"""
        if self.current_dron:
            self.current_dron.destroy()

        positions = {
            'Front': {'x': 240, 'y': 160},
            'Right': {'x': 350, 'y': 320},
            'Left': {'x': 120, 'y': 320},
            'Back': {'x': 250, 'y': 500},
            'Up': {'x': 200, 'y': 100}
        }

        if direction in positions and self.dron_img:
            pos = positions[direction]
            self.current_dron = Label(self.root, image=self.dron_img, bg='red')
            self.current_dron.place(x=pos['x'], y=pos['y'])

    def danger(self, many):
        """Отображает предупреждение об обнаруженных дронах"""
        if hasattr(self, 'danger_label'):
            self.danger_label.destroy()
        if hasattr(self, 'status_label'):
            self.status_label.destroy()
        
        try:
            self.danger_label = create_image(self.root, 'dangeros.png', x=380, y=10, width=150, bg='snow')
            
            if many == 1:
                text = f'Обнаружен {many} FPV-дрон!'
            elif 2 <= many <= 4:
                text = f'Обнаружено {many} FPV-дрона!'
            else:
                text = f'Обнаружено {many} FPV-дронов!'
            
            self.status_label = create_text_label(
                self.root,
                text=text,
                x=325, y=100,
                font_size=14,
                bg='red',
                fg='white',
                font_weight='bold'
            )
        except Exception as e:
            print(f"Ошибка при создании элементов опасности: {e}")

# Класс виджета камеры с детектором дронов
class CameraWidget:
    def __init__(self, parent, x=0, y=0):
        self.parent = parent
        self.camera_active = False
        self.video_capture = None
        self.drone_detected = False
        self.drone_count = 0
        
        # Загрузка модели YOLOv12n
        try:
            self.model = YOLO('best_drone_detections_yolov12n.pt')
            self.model.conf = 0.5  # Порог уверенности
            print("Модель успешно загружена")
        except Exception as e:
            print(f"Ошибка загрузки модели: {e}")
            raise
        
        # Создание интерфейса
        self.camera_frame = Frame(parent, bg='green')
        self.camera_frame.place(x=x, y=y)
        
        self.camera_label = Label(self.camera_frame, bg='black')
        self.camera_label.pack()
        
        self.btn_camera = Button(self.camera_frame, 
                               text='Включить камеру', 
                               command=self.toggle_camera,
                               font=('Arial', 10),
                               bg='blue',
                               fg='white')
        self.btn_camera.pack(pady=5)
        
        self.signals = Signals(parent)
    
    def toggle_camera(self):
        """Включение/выключение камеры"""
        if not self.camera_active:
            self.start_camera()
        else:
            self.stop_camera()
    
    def start_camera(self):
        """Запуск камеры"""
        self.camera_active = True
        self.btn_camera.config(text='Выключить камеру', bg='red')
        self.video_capture = cv2.VideoCapture(0)
        if not self.video_capture.isOpened():
            print("Ошибка: Не удалось открыть камеру")
            return
        Thread(target=self.update_frame, daemon=True).start()
    
    def stop_camera(self):
        """Остановка камеры"""
        self.camera_active = False
        self.btn_camera.config(text='Включить камеру', bg='blue')
        if self.video_capture:
            self.video_capture.release()
        self.camera_label.config(image='')
        if hasattr(self.signals, 'current_dron'):
            self.signals.current_dron.destroy()
        if hasattr(self.signals, 'danger_label'):
            self.signals.danger_label.destroy()
    
    def detect_drones(self, frame):
        """Детекция дронов на кадре с помощью YOLOv12n"""
        try:
            results = self.model(frame, verbose=False)
            drone_count = 0
            
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    if box.conf.item() >= self.model.conf:
                        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, f"Drone {box.conf.item():.2f}", 
                                   (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        drone_count += 1
            
            return drone_count, frame
        except Exception as e:
            print(f"Ошибка детекции: {e}")
            return 0, frame
    
    def update_frame(self):
        """Обновление кадров с детекцией"""
        while self.camera_active:
            ret, frame = self.video_capture.read()
            if not ret:
                print("Ошибка: Не удалось получить кадр с камеры")
                break
            
            try:
                drone_count, processed_frame = self.detect_drones(frame)
                
                if drone_count > 0:
                    if not self.drone_detected or self.drone_count != drone_count:
                        self.drone_detected = True
                        self.drone_count = drone_count
                        self.signals.dron_direction('Front')
                        self.signals.danger(drone_count)
                else:
                    if self.drone_detected:
                        self.drone_detected = False
                        if hasattr(self.signals, 'current_dron'):
                            self.signals.current_dron.destroy()
                        if hasattr(self.signals, 'danger_label'):
                            self.signals.danger_label.destroy()
                
                processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                processed_frame = cv2.resize(processed_frame, (180, 140))
                img = Image.fromarray(processed_frame)
                imgtk = ImageTk.PhotoImage(image=img)
                
                self.camera_label.imgtk = imgtk
                self.camera_label.config(image=imgtk)
                
                self.parent.update()
                time.sleep(0.03)
            except Exception as e:
                print(f"Ошибка обработки кадра: {e}")
                break

class Signals:
    def __init__(self, root):
        self.root = root
        self.current_dron = None  # Текущее изображение дрона
        self.dron_img = self._load_image()  # Загружаем изображение один раз

    def _load_image(self):
        """Загружает и подготавливает изображение дрона"""
        try:
            img = Image.open('dron_atak.png')
            img = img.resize((70, 70), Image.LANCZOS)  # Фиксированный размер
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Ошибка загрузки изображения дрона: {e}")
            return None

    def dron_direction(self, direction):
        """Отображает атаку дрона с указанного направления"""
        # Удаляем предыдущее изображение
        if self.current_dron:
            self.current_dron.destroy()

        # Позиции для разных направлений
        positions = {
            'Front': {'x': 240, 'y': 160},
            'Right': {'x': 350, 'y': 320},
            'Left': {'x': 120, 'y': 320},
            'Back': {'x': 250, 'y': 500},
            'Up': {'x': 200, 'y': 100}
        }

        # Создаем новое изображение дрона
        if direction in positions and self.dron_img:
            pos = positions[direction]
            self.current_dron = Label(self.root, image=self.dron_img, bg='red')
            self.current_dron.place(x=pos['x'], y=pos['y'])

    def danger(self, many):
        """
        Отображает предупреждение об обнаруженных FPV-дронах
        
        Параметры:
            many: количество обнаруженных дронов (int)
        """
        # Удаляем предыдущие элементы, если они есть
        if hasattr(self, 'danger_label'):
            self.danger_label.destroy()
        if hasattr(self, 'fpv_label'):
            self.fpv_label.destroy()
        if hasattr(self, 'status_label'):
            self.status_label.destroy()
        
        # Создаем элементы предупреждения
        try:
            self.danger_label = create_image(self.root, 'dangeros.png', x=380, y=10, width=150, bg='snow')
            #self.fpv_label = create_image(self.root, 'fpv.jpeg', x=500, y=50, width=100)

            if many == 1:
                Text=f'Обнаружен {many} FPV-дрон!'
            elif 2<= many <=4:
                Text=f'Обнаружено {many} FPV-дрона!'
            else:
                Text=f'Обнаружено {many} FPV-дронов!'
            
            self.status_label = create_text_label(
                self.root,
                text=Text,
                x=325, y=100,
                font_size=14,
                bg='red',
                fg='white',
                font_weight='bold'
            )
        except Exception as e:
            print(f"Ошибка при создании элементов опасности: {e}")    


# Проверка наличия необходимых файлов
required_files = [
    'best_drone_detections_yolov12n.pt',
    'tank.png',
    'dron_atak.png',
    'dangeros.png',
    'icon.ico'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print(f"Предупреждение: Отсутствуют необходимые файлы: {', '.join(missing_files)}")

# Создание и запуск главного окна
root = Tk()
root.geometry('600x600')
root.resizable(False, False)
root.title('Система Щит')
root.config(bg='green')

try:
    root.iconbitmap('icon.ico')
except:
    print("Не удалось загрузить иконку приложения")

# Виджет камеры
camera = CameraWidget(root, x=0, y=0)

# Изображение танка
tank = create_image(root, 'tank.png', x=150, y=150, width=300, bg='green')

root.mainloop()

Модель успешно загружена


In [ ]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np
from threading import Thread
from ultralytics import YOLO
import time
from tkinter import font as tkfont
import os

def create_image(root, filename, x=0, y=0, width=None, height=None, bg=None):
    """
    Создает и размещает изображение в окне Tkinter
    
    Параметры:
        root: окно Tkinter (Tk() или Frame)
        filename: имя файла изображения (например 'tank.png')
        x, y: координаты размещения (по умолчанию 0, 0)
        width: желаемая ширина (None - оригинальный размер)
        height: желаемая высота (None - оригинальный размер)
        bg: цвет фона (None - прозрачный)
    
    Возвращает:
        Label: созданный виджет с изображением
    """
    # Загружаем изображение через Pillow
    pil_image = Image.open(filename)
    
    # Изменяем размер если нужно
    if width or height:
        original_width, original_height = pil_image.size
        
        # Если указана только одна размерность, сохраняем пропорции
        if not width:
            ratio = height / original_height
            width = int(original_width * ratio)
        elif not height:
            ratio = width / original_width
            height = int(original_height * ratio)
            
        pil_image = pil_image.resize((width, height), Image.LANCZOS)
    
    # Конвертируем в формат Tkinter
    tk_image = ImageTk.PhotoImage(pil_image)
    
    # Создаем Label с изображением
    label = Label(root, image=tk_image, bg=bg)
    label.image = tk_image  # Сохраняем ссылку на изображение!
    label.place(x=x, y=y)
    
    return label

from tkinter import font as tkfont  # Для работы с шрифтами

def create_text_label(root, text, x=0, y=0, 
                     font_size=12, font_family='Arial', font_weight='normal',
                     bg='white', fg='black', 
                     width=None, height=None,
                     anchor='nw'):
    """
    Создает текстовую метку с заданными параметрами
    
    Параметры:
        root: окно Tkinter (Tk или Frame)
        text: отображаемый текст
        x, y: координаты размещения (по умолчанию 0, 0)
        font_size: размер шрифта (по умолчанию 12)
        font_family: семейство шрифтов (по умолчанию Arial)
        font_weight: насыщенность (normal/bold)
        bg: цвет фона (по умолчанию white)
        fg: цвет текста (по умолчанию black)
        width: ширина метки (None - автоматически)
        height: высота метки (None - автоматически)
        anchor: выравнивание текста (nw, n, ne, w, center, e, sw, s, se)
    
    Возвращает:
        Label: созданный текстовый виджет
    """
    # Создаем объект шрифта
    custom_font = tkfont.Font(
        family=font_family,
        size=font_size,
        weight=font_weight
    )
    
    # Создаем метку
    label = Label(
        root,
        text=text,
        font=custom_font,
        bg=bg,
        fg=fg,
        width=width,
        height=height,
        anchor=anchor
    )
    
    # Размещаем метку
    label.place(x=x, y=y)
    
    return label


# Класс для работы с сигналами
class Signals:
    def __init__(self, root):
        self.root = root
        self.current_dron = None
        self.dron_img = self._load_image()

    def _load_image(self):
        """Загружает изображение дрона"""
        try:
            img = Image.open('dron_atak.png')
            img = img.resize((70, 70), Image.LANCZOS)
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Ошибка загрузки изображения дрона: {e}")
            return None

    def dron_direction(self, direction):
        """Отображает атаку дрона с указанного направления"""
        if self.current_dron:
            self.current_dron.destroy()

        positions = {
            'Front': {'x': 240, 'y': 160},
            'Right': {'x': 350, 'y': 320},
            'Left': {'x': 120, 'y': 320},
            'Back': {'x': 250, 'y': 500},
            'Up': {'x': 200, 'y': 100}
        }

        if direction in positions and self.dron_img:
            pos = positions[direction]
            self.current_dron = Label(self.root, image=self.dron_img, bg='red')
            self.current_dron.place(x=pos['x'], y=pos['y'])

    def danger(self, many):
        """Отображает предупреждение об обнаруженных дронах"""
        if hasattr(self, 'danger_label'):
            self.danger_label.destroy()
        if hasattr(self, 'status_label'):
            self.status_label.destroy()
        
        try:
            self.danger_label = create_image(self.root, 'dangeros.png', x=380, y=10, width=150, bg='snow')
            
            if many == 1:
                text = f'Обнаружен {many} FPV-дрон!'
            elif 2 <= many <= 4:
                text = f'Обнаружено {many} FPV-дрона!'
            else:
                text = f'Обнаружено {many} FPV-дронов!'
            
            self.status_label = create_text_label(
                self.root,
                text=text,
                x=325, y=100,
                font_size=14,
                bg='red',
                fg='white',
                font_weight='bold'
            )
        except Exception as e:
            print(f"Ошибка при создании элементов опасности: {e}")

# Класс виджета камеры с детектором дронов
class CameraWidget:
    def __init__(self, parent, x=0, y=0):
        self.parent = parent
        self.camera_active = False
        self.video_capture = None
        self.drone_detected = False
        self.drone_count = 0
        
        # Загрузка модели YOLOv12n
        try:
            self.model = YOLO('best_drone_detections_yolov12n.pt')
            self.model.conf = 0.5  # Порог уверенности
            print("Модель успешно загружена")
        except Exception as e:
            print(f"Ошибка загрузки модели: {e}")
            raise
        
        # Создание интерфейса
        self.camera_frame = Frame(parent, bg='green')
        self.camera_frame.place(x=x, y=y)
        
        self.camera_label = Label(self.camera_frame, bg='black')
        self.camera_label.pack()
        
        self.btn_camera = Button(self.camera_frame, 
                               text='Включить камеру', 
                               command=self.toggle_camera,
                               font=('Arial', 10),
                               bg='blue',
                               fg='white')
        self.btn_camera.pack(pady=5)
        
        self.signals = Signals(parent)
    
    def toggle_camera(self):
        """Включение/выключение камеры"""
        if not self.camera_active:
            self.start_camera()
        else:
            self.stop_camera()
    
    def start_camera(self):
        """Запуск камеры"""
        self.camera_active = True
        self.btn_camera.config(text='Выключить камеру', bg='red')
        self.video_capture = cv2.VideoCapture(0)
        if not self.video_capture.isOpened():
            print("Ошибка: Не удалось открыть камеру")
            return
        Thread(target=self.update_frame, daemon=True).start()
    
    def stop_camera(self):
        """Остановка камеры"""
        self.camera_active = False
        self.btn_camera.config(text='Включить камеру', bg='blue')
        if self.video_capture:
            self.video_capture.release()
        self.camera_label.config(image='')
        if hasattr(self.signals, 'current_dron'):
            self.signals.current_dron.destroy()
        self.signals.clear_danger_signals()  # Удаляем все элементы предупреждения
    
    def detect_drones(self, frame):
        """Детекция дронов на кадре с помощью YOLOv12n"""
        try:
            results = self.model(frame, verbose=False)
            drone_count = 0
            
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    if box.conf.item() >= self.model.conf:
                        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, f"Drone {box.conf.item():.2f}", 
                                   (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        drone_count += 1
            
            return drone_count, frame
        except Exception as e:
            print(f"Ошибка детекции: {e}")
            return 0, frame
    
    def update_frame(self):
        """Обновление кадров с детекцией"""
        while self.camera_active:
            ret, frame = self.video_capture.read()
            if not ret:
                print("Ошибка: Не удалось получить кадр с камеры")
                break
            
            try:
                drone_count, processed_frame = self.detect_drones(frame)
                
                if drone_count > 0:
                    if not self.drone_detected or self.drone_count != drone_count:
                        self.drone_detected = True
                        self.drone_count = drone_count
                        self.signals.dron_direction('Front')
                        self.signals.danger(drone_count)
                else:
                    if self.drone_detected:
                        self.drone_detected = False
                        if hasattr(self.signals, 'current_dron'):
                            self.signals.current_dron.destroy()
                        if hasattr(self.signals, 'danger_label'):
                            self.signals.danger_label.destroy()
                
                processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                processed_frame = cv2.resize(processed_frame, (180, 140))
                img = Image.fromarray(processed_frame)
                imgtk = ImageTk.PhotoImage(image=img)
                
                self.camera_label.imgtk = imgtk
                self.camera_label.config(image=imgtk)
                
                self.parent.update()
                time.sleep(0.03)
            except Exception as e:
                print(f"Ошибка обработки кадра: {e}")
                break

class Signals:
    def __init__(self, root):
        self.root = root
        self.current_dron = None  # Текущее изображение дрона
        self.dron_img = self._load_image()  # Загружаем изображение один раз

    def _load_image(self):
        """Загружает и подготавливает изображение дрона"""
        try:
            img = Image.open('dron_atak.png')
            img = img.resize((70, 70), Image.LANCZOS)  # Фиксированный размер
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Ошибка загрузки изображения дрона: {e}")
            return None

    def dron_direction(self, direction):
        """Отображает атаку дрона с указанного направления"""
        # Удаляем предыдущее изображение
        if self.current_dron:
            self.current_dron.destroy()

        # Позиции для разных направлений
        positions = {
            'Front': {'x': 240, 'y': 160},
            'Right': {'x': 350, 'y': 320},
            'Left': {'x': 120, 'y': 320},
            'Back': {'x': 250, 'y': 500},
            'Up': {'x': 200, 'y': 100}
        }

        # Создаем новое изображение дрона
        if direction in positions and self.dron_img:
            pos = positions[direction]
            self.current_dron = Label(self.root, image=self.dron_img, bg='red')
            self.current_dron.place(x=pos['x'], y=pos['y'])

    def danger(self, many):
    """
    Отображает предупреждение об обнаруженных FPV-дронах
    
    Параметры:
        many: количество обнаруженных дронов (int)
    """
    # Удаляем предыдущие элементы
    self.clear_danger_signals()
    
    # Если дронов нет (many=0), просто выходим
    if many == 0:
        return
        
    # Создаем новые элементы предупреждения
    try:
        # Изображение опасности
        self.danger_label = create_image(self.root, 'dangeros.png', x=380, y=10, width=150, bg='snow')
        
        # Текст с количеством дронов
        if many == 1:
            text = f'Обнаружен {many} FPV-дрон!'
        elif 2 <= many <= 4:
            text = f'Обнаружено {many} FPV-дрона!'
        else:
            text = f'Обнаружено {many} FPV-дронов!'
        
        self.status_label = create_text_label(
            self.root,
            text=text,
            x=325, y=100,
            font_size=14,
            bg='red',
            fg='white',
            font_weight='bold'
        )
        
        # Сохраняем ссылки на созданные элементы
        self.danger_elements = [self.danger_label, self.status_label]
        
    except Exception as e:
        print(f"Ошибка при создании элементов опасности: {e}")

def clear_danger_signals(self):
    """Удаляет все элементы предупреждения об опасности"""
    if hasattr(self, 'danger_elements'):
        for element in self.danger_elements:
            try:
                if element and hasattr(element, 'destroy'):
                    element.destroy()
            except Exception as e:
                print(f"Ошибка при удалении элемента: {e}")
        del self.danger_elements
    
    # Дополнительные проверки для обратной совместимости
    for attr in ['danger_label', 'fpv_label', 'status_label']:
        if hasattr(self, attr):
            try:
                getattr(self, attr).destroy()
            except:
                pass
            delattr(self, attr)


# Проверка наличия необходимых файлов
required_files = [
    'best_drone_detections_yolov12n.pt',
    'tank.png',
    'dron_atak.png',
    'dangeros.png',
    'icon.ico'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print(f"Предупреждение: Отсутствуют необходимые файлы: {', '.join(missing_files)}")

# Создание и запуск главного окна
root = Tk()
root.geometry('600x600')
root.resizable(False, False)
root.title('Система Щит')
root.config(bg='green')

try:
    root.iconbitmap('icon.ico')
except:
    print("Не удалось загрузить иконку приложения")

# Виджет камеры
camera = CameraWidget(root, x=0, y=0)

# Изображение танка
tank = create_image(root, 'tank.png', x=150, y=150, width=300, bg='green')

root.mainloop()